<a href="https://colab.research.google.com/github/ShubhankarSah/AIInternAssignment/blob/main/Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install all necessary dependencies
!pip install ultralytics
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -U pip setuptools
!pip install -U opencv-python-headless
!pip install -U matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.8/874.8 kB 14.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:13 https://develope

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 98.7 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3367, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3147, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 105, in _run_wrapper
    status = _inner_run()
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 96, in _inner_run
    return self.run(options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/di

In [2]:
!pip install fer


  Using cached fer-22.5.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached facenet_pytorch-2.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached ffmpeg-1.4.tar.gz (5.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached pillow-10.2.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
  Using cached torch-2.2.2-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached torchvision-0.17.2-cp310-cp310-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_

In [2]:
!pip install deepface

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 42.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=627bce24b38838ea8d07e9cb8dbfa6e8e18a496bb332eccc72426ab27ff5d11f
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [4]:
!pip install gender-guesser


In [1]:
# Import necessary libraries
import cv2
import os
import torch
import pytesseract
import pandas as pd
from ultralytics import YOLO  # YOLOv8 import
from PIL import Image
from base64 import b64encode
from IPython.display import display, HTML
from google.colab import files

# Function to upload an image in Colab
def upload_image():
    uploaded = files.upload()
    if uploaded:
        for filename in uploaded.keys():
            return filename
    return None

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # Use the YOLOv8 model

# Function to preprocess the image for better OCR accuracy
def preprocess_for_ocr(img):
    # Convert to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply thresholding
    _, binary_img = cv2.threshold(gray_img, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary_img

# Function to detect text using Tesseract
def detect_text(img):
    preprocessed_img = preprocess_for_ocr(img)
    text = pytesseract.image_to_string(preprocessed_img)
    return text.strip()

# Function to extract text regions and save them as images
def extract_and_save_text_regions(img, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    preprocessed_img = preprocess_for_ocr(img)
    boxes = pytesseract.image_to_boxes(preprocessed_img)

    text_objects = []
    h, w = img.shape[:2]  # Image dimensions
    for i, box in enumerate(boxes.splitlines()):
        box = box.split(' ')
        x1, y1, x2, y2 = int(box[1]), int(box[2]), int(box[3]), int(box[4])
        text_img = img[h - y2:h - y1, x1:x2]  # Crop the text region

        if text_img.size != 0:  # Ensure the image is not empty
            text_filename = f"{output_dir}/text_{i}.png"
            cv2.imwrite(text_filename, text_img)  # Save cropped text region

            # Save detected text as an object
            text_objects.append({
                "Object ID": f"Text_{i}",
                "Description": f"Detected text: {box[0]}",
                "Image": text_filename
            })

    return text_objects

# Function to save object images and provide descriptions
def save_object_images(results, img, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    object_data = []

    # YOLOv8 returns results as a list, so we iterate over each detection result
    for result in results:
        boxes = result.boxes.xyxy  # Bounding box coordinates
        class_ids = result.boxes.cls  # Class IDs
        confs = result.boxes.conf  # Confidence scores

        for i, (box, class_id, conf) in enumerate(zip(boxes, class_ids, confs)):
            class_name = result.names[int(class_id)]  # Class name from YOLO model

            # Crop detected object
            x1, y1, x2, y2 = map(int, box)  # Bounding box coordinates
            object_img = img[y1:y2, x1:x2]

            if object_img.size != 0:  # Ensure the cropped image is not empty
                object_filename = f"{output_dir}/{class_name}_{i}.png"
                cv2.imwrite(object_filename, object_img)  # Save the object image

                # Basic description of the object
                description = f"Detected a {class_name} with confidence {conf:.2f}"

                # Store the object data
                object_data.append({
                    "Object ID": class_name,
                    "Description": description,
                    "Image": object_filename
                })

    return object_data

# Function to summarize image content
def summarize_image(img_path, output_dir="output"):
    img = cv2.imread(img_path)

    # Check if the image was loaded correctly
    if img is None:
        print(f"Error: Could not load image at {img_path}. Please check the file path.")
        return

    # Perform object detection with YOLOv8
    results = model(img)

    # Save detected objects and get object data
    object_data = save_object_images(results, img, output_dir)

    # Detect and save text in the image using Tesseract
    text_data = extract_and_save_text_regions(img, output_dir)
    object_data.extend(text_data)

    # Create a summary table and save it as CSV
    for idx, obj in enumerate(object_data, start=1):
        obj['S.No.'] = idx  # Add serial number

    df = pd.DataFrame(object_data)
    df.to_csv(f"{output_dir}/summary.csv", index=False)

    # Display the summary table with images
    html = "<table><tr><th>S.No.</th><th>Object ID</th><th>Description</th><th>Image</th></tr>"
    for row in object_data:
        if row['Image'] != "Text Detected":
            with open(row["Image"], "rb") as img_file:
                img_data = b64encode(img_file.read()).decode("utf-8")
                img_html = f'<img src="data:image/png;base64,{img_data}" width="100"/>'
        else:
            img_html = "Text Detected"
        html += f"<tr><td>{row['S.No.']}</td><td>{row['Object ID']}</td><td>{row['Description']}</td><td>{img_html}</td></tr>"
    html += "</table>"
    display(HTML(html))

# Step 1: Upload the image in Google Colab
img_path = upload_image()

# Step 2: Run the summarization on the uploaded image
if img_path:
    summarize_image(img_path)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 89.0MB/s]


Saving pexels-cottonbro-4056475.jpg to pexels-cottonbro-4056475 (1).jpg

0: 448x640 2 persons, 2 dogs, 1 bowl, 1 potted plant, 2 books, 368.4ms
Speed: 18.8ms preprocess, 368.4ms inference, 39.4ms postprocess per image at shape (1, 3, 448, 640)
